In [54]:
from pysmt.shortcuts import Symbol, And, GE, LE, Int, Solver, Equals, Implies, Or, NotEquals, Not, Iff, Plus, Ite
from pysmt.typing import INT
from utils import *
from SMT_utils import *

In [55]:
##### Leggiamo un'istanza ####
instance_num = 1

instance = load_instance(dir="/home/riccardo/Scrivania/CDMO/input", instance_num=instance_num)

In [56]:
exactly_one = exactly_one_he

In [57]:
def set_constraints(instance, solver):
    # prendiamo i dati in ingresso dell'istanza
    m, n, l, s, D = instance.get_values()


    # DECISION VARIABLES

    # X_i_k = j  <->  il corriere i è in posizione j al momento k
    # i = 0,..., m-1 (corrieri)
    # k = 0,..., n+1 (momenti temporali)
    # j = 1,..., n+1 dove n+1 è la base (posizione)
    # X = [[Symbol(f"x_{i}_{k}", INT) for i in range(m)] for k in range(n+2)]
    X = [[Symbol(f"x_{i}_{k}", INT) for k in range(n+2)] for i in range(m)]


    # carried_load[i] terrà il peso totale portato dal corriere i-esimo
    carried_load = [Symbol(f"carried_load_{i}", INT) for i in range(m)]

    # traveled_distance[i] terrà la distanza totale percorsa dal corriere i-esimo
    traveled_distance = [Symbol(f"traveled_distance_{i}", INT) for i in range(m)]

    # obj_function sarà il valore massimo tra tutte le distanze percorse
    obj_function = Symbol(f"obj_function", INT)


    # definiamo il dominio di X, j = 1,..., n+1 dove n+1 è la base (posizione)
    domain_X = []
    for row in X:
        row_domain = And([And(GE(x, Int(1)),
                                LE(x, Int(n+1))) for x in row])
        domain_X.append(row_domain)

    # imponiamo il dominio di X al solver
    solver.add_assertion(And(domain_X))



    # TUTTI I CORRIERI DEVONO INIZIARE IN BASE (al momento k=0 il corriere è in j=n+1)
    for i in range(m):
        solver.add_assertion(Equals(X[i][0], Int(n+1)))


    # TUTTI I CORRIERI PRIMA O POI DEVONO TORNARE IN BASE (in un momento k>=1 il corriere dev'essere almeno una volta in j=n+1)
    for i in range(m):
        back_row_constr = Or(
            [Equals(X[i][k], Int(n+1)) for k in range(1, n+2)]
        )
        solver.add_assertion(back_row_constr)
   
    
    # SE UN CORRIERE È IN BASE AL MOMENTO K ALLORA LO SARÀ ANCHE AL MOMENTO K+1 (esluso il momento 0)
    for i in range(m):
        for k in range(1, n+1):
            solver.add_assertion(
                                Implies(
                                    Equals(X[i][k], Int(n+1)),
                                    Equals(X[i][k+1], Int(n+1))
                                )
            )

    
    # TUTTI I LUOGHI DEVONO ESSERE VISITATI SOLO UNA VOLTA
    for j in range(1, n+1):
        assertions = [Equals(X[i][k], Int(j)) for i in range(m) for k in range(1, n + 1)]
        solver.add_assertion(exactly_one(assertions, f"one_visit_{j}"))
    

    
    # carried_load[i] avrà la somma di tutti i pesi trasportati dal corrieri i
    for i in range(m):
        # In implications metto le implicazioni che uso per calcolare le somme dei pesi
        implications = []
        for j in range(1, n+1):
            # Metto qui la somma per ogni posizione visitata
            # Se X[i][k] è uguale a j allora in Plus metto s[j-1], altrimenti metto 0
            # In sum_expr metto la somma di tutti i pesi trasportati
            sum_expr = Plus([Ite(Equals(X[i][k], Int(j)),
                                 Int(s[j - 1]), 
                                 Int(0)) for k in range(1, n+1)])
            
            implications.append(sum_expr)

        # Aggiungi l'assertion al solver: carried_load[i] = somma dei pesi trasportati
        solver.add_assertion(Equals(carried_load[i], Plus(implications)))



    # OGNI CORRIERE NON PUÒ TRASPORTARE PIÙ DI UN CERTO PESO
    for i in range(m):
        solver.add_assertion(LE(carried_load[i], Int(l[i])))


    
    # traveled_distances[i] avrà la somma delle distanza percorse dal corriere i
    for i in range(m):
        distances = []

        # prendo la distanza dalla posizione iniziale
        for j in range(1, n+2):
            first_distance = Plus([Ite(Equals(X[i][1], Int(j)),
                                        Int(int(D[n][j-1])),
                                        Int(0))])
            distances.append(first_distance)
            
        # prendo le distanze tra due posizioni visitate consecutivamente
        for k in range(1, n+1):

            for j1 in range(1, n + 2):
                for j2 in range(1, n + 2):
                    distances.append(Plus([Ite(And(Equals(X[i][k], Int(j1)),
                                                Equals(X[i][k+1], Int(j2))),
                                                Int(int(D[j1-1][j2-1])),
                                                Int(0)
                                            )]))

        solver.add_assertion(Equals(traveled_distance[i], Plus(distances)))


    # impongo che la obj_function sia il valore maggiore tra le distanze percorse dai corrieri
    for i in range(m):
        solver.add_assertion(GE(obj_function, traveled_distance[i]))


    # return variables
    return X, carried_load, traveled_distance, obj_function

In [53]:
with Solver(name='z3') as solver:
    
    X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)
    
    found_sol = solver.solve()

    if found_sol:
        print("#### Soluzione trovata #####\n")
        # prendo tutte le variabili x_i_k
        for i in range(instance.m):
            print(f"Percorso del corriere numero {i}:")
            for k in X[i]:
                print(f"{k}: {solver.get_value(k)}")
            print(f"Peso portato dal corriere numero {i}:")
            print(f"{solver.get_value(carried_load[i])}")
            print(f"Distanza totale percorsa dal corriere {i}:")
            print(f"{solver.get_value(traveled_distance[i])}")
        print(f"Quindi, la funzione obiettivo è: {solver.get_value(obj_function)}")
    else:
        print("Nessuna soluzione trovata.")

#### Soluzione trovata #####

Percorso del corriere numero 0:
x_0_0: 4
x_0_1: 2
x_0_2: 4
x_0_3: 4
x_0_4: 4
Peso portato dal corriere numero 0:
17
Distanza totale percorsa dal corriere 0:
160
Percorso del corriere numero 1:
x_1_0: 4
x_1_1: 1
x_1_2: 3
x_1_3: 4
x_1_4: 4
Peso portato dal corriere numero 1:
26
Distanza totale percorsa dal corriere 1:
206
Quindi, la funzione obiettivo è: 206
